# Experiment with multiple and more complex conditions

In [1]:
import numpy as np
from regressor import LinearRegressor

from inversion_ideas import (
    ChiTarget,
    ConjugateGradient,
    DataMisfit,
    Inversion,
    ModelChanged,
    MultiplierCooler,
    ObjectiveChanged,
    TikhonovZero,
)

## Create a true model and synthetic data for a linear regressor

In [2]:
n_params = 10
rng = np.random.default_rng(seed=4242)
true_model = rng.uniform(size=10)
true_model

array([0.78225148, 0.67148671, 0.2373809 , 0.17946133, 0.34662367,
       0.15210999, 0.31142952, 0.23900652, 0.54355731, 0.91770851])

In [3]:
# Build the X array
n_data = 25
shape = (n_data, n_params)
X = rng.uniform(size=n_data * n_params).reshape(shape)

In [4]:
synthetic_data = X @ true_model
maxabs = np.max(np.abs(synthetic_data))
noise = rng.normal(scale=1e-2 * maxabs, size=synthetic_data.size)
synthetic_data += noise
synthetic_data

array([2.83840696, 2.18091081, 2.00623242, 2.08333039, 2.01694883,
       2.7826232 , 2.10564027, 1.27333506, 2.08859855, 1.94177648,
       1.88492037, 2.92394733, 2.17231952, 3.08009275, 1.61670886,
       1.77403753, 2.67305005, 1.91413882, 2.42117827, 2.13991628,
       2.0153805 , 2.71388471, 2.65944255, 2.44416121, 3.14217523])

## Inversion

In [5]:
uncertainty = 1e-2 * maxabs * np.ones_like(synthetic_data)
simulation = LinearRegressor(X, sleep=1)
data_misfit = DataMisfit(synthetic_data, uncertainty, simulation)

In [6]:
smallness = TikhonovZero(n_params)

In [7]:
# Objective function
beta_0 = 1e4
regularization = beta_0 * smallness
phi = data_misfit + regularization

# Initial model
initial_model = np.zeros(n_params)

# Minimizer
minimizer = ConjugateGradient()

# Directives
beta_cooler = MultiplierCooler(regularization, cooling_factor=2.0)

# Stopping criteria
chi_target = ChiTarget(data_misfit, chi_target=1.0)
model_changed = ModelChanged(rtol=1e-1)
phi_changed = ObjectiveChanged(phi, rtol=1e-1)
stopping_criteria = chi_target | model_changed | phi_changed

In [8]:
inversion = Inversion(
    phi,
    initial_model,
    minimizer,
    directives=[beta_cooler],
    stopping_criteria=stopping_criteria,
    cache_models=True,
)

In [9]:
inverted_model = inversion.run()

Output()

INFO: 🎉 Inversion successfully finished due to stopping criteria.


In [10]:
stopping_criteria.info(inverted_model)

[x] LogicalOr
├── [x] LogicalOr
│   ├── ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│   │   │ [ ] ChiTarget                                                                                           │
│   │   │ ├── Condition: chi < chi_target                                                                         │
│   │   │ ├── chi        = 9.23e+00                                                                               │
│   │   │ └── chi_target = 1.00e+00                                                                               │
│   │   ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
│   └── ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────╮
│       │ [x] ModelChanged                                                                                        │
│       │ ├── Condition: |m - m_prev| <= max(|m_prev| * rtol, atol)                                               │
│       │ ├── |m - m_prev| = 1.32e-01                                                                             │
│       │ ├── |m_prev|     = 1.41e+00                                                                             │
│       │ ├── rtol         = 1.00e-01                                                                             │
│       │ └── atol         = 0.00e+00                                                                             │
│       ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────╯
└── ╭─────────────────────────────────────────────────────────────────────────────────────────────────────────────╮
    │ [ ] ObjectiveChanged                                                                                        │
    │ ├── Condition: |φ(m) - φ(m_prev)| <= max(|φ(m_prev)| * rtol, atol)                                          │
    │ ├── |φ(m) - φ(m_prev)| = 2.59e+03                                                                           │
    │ ├── |φ(m_prev)|        = 5.55e+03                                                                           │
    │ ├── rtol               = 1.00e-01                                                                           │
    │ └── atol               = 0.00e+00                                                                           │
    ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [11]:
tree = chi_target.info(inverted_model)
tree

[ ] ChiTarget
├── Condition: chi < chi_target
├── chi        = 9.23e+00
└── chi_target = 1.00e+00

In [12]:
chi_target(inverted_model)

False

In [13]:
model_changed(inverted_model)

True

In [14]:
phi_changed(inverted_model)

np.False_

## Reuse conditions

If we reuse the conditions, the new inversion should initialize the stopping criteria on the zeroth inversion, starting from fresh.

In [15]:
# Reset starting beta in the regularization
regularization.multiplier = 1e4

inversion = Inversion(
    phi,
    initial_model,
    minimizer,
    directives=[beta_cooler],
    stopping_criteria=stopping_criteria,
    cache_models=True,
)

In [16]:
inverted_model = inversion.run()

Output()

INFO: 🎉 Inversion successfully finished due to stopping criteria.
